In [ ]:
!pip install vk_api > None

In [ ]:
from vk_api import VkApi
from vk_api.utils import get_random_id
import requests
import json
import pandas as pd
from tqdm import trange
import time
import numpy as np
import requests
import warnings
#warnings.simplefilter("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#вставить ваш ключ доступа
VK_TOKEN = ''

Функция сбора данных поста вк, чтобы проверить работу вк api

In [ ]:
owner_id = '-15755094', # id сообщества указывается со знаком - перед цифрами
offset = 0, #смещение (сколько постов надо пропустить, чтобы начать сбор)
count = 1 # сколько постов получить. максимум можно получить 100
v = 5.199

def get_wall_posts(VK_TOKEN: str, owner_id: str, offset: int, count: int, v: float) -> dict:
    url = 'https://api.vk.com/method/wall.get' #метод wall.get позволяет получать записи со стены
    params = {
        'access_token': VK_TOKEN, # ваш токен
        'owner_id': owner_id,
        'v': v,
        'offset': offset,
        'count': count
    }

    response = requests.get(url, params=params)
    return response.json()

wall_posts = get_wall_posts(VK_TOKEN, owner_id, offset, count, v)
print(wall_posts)

{'response': {'count': 243835, 'items': [{'inner_type': 'wall_wallpost', 'donut': {'is_donut': False}, 'comments': {'can_post': 1, 'can_view': 1, 'count': 6, 'groups_can_post': True}, 'marked_as_ads': 0, 'hash': 'KAwAIC5-ksrXJaVqrg', 'type': 'post', 'push_subscription': {'is_subscribed': False}, 'attachments': [{'type': 'photo', 'photo': {'album_id': -7, 'date': 1740081386, 'id': 459710959, 'owner_id': -15755094, 'access_key': 'ba7e02ff72b5654193', 'post_id': 47349788, 'sizes': [{'height': 71, 'type': 's', 'width': 75, 'url': 'https://sun1-14.userapi.com/s/v1/ig2/h-z1FUvX4M7HOz6AXYg7CpqnFeRQoUBZNp_kgHbGtCF0qr7qN-UX1r3EOQPkarUp0Y7abj-gVypjSRJMOPNH-9HS.jpg?quality=95&as=32x30,48x46,72x69,108x103,160x152,240x228,360x343,480x457,540x514,640x609,720x685,1080x1028,1100x1047&from=bu&cs=75x71'}, {'height': 124, 'type': 'm', 'width': 130, 'url': 'https://sun1-14.userapi.com/s/v1/ig2/h-z1FUvX4M7HOz6AXYg7CpqnFeRQoUBZNp_kgHbGtCF0qr7qN-UX1r3EOQPkarUp0Y7abj-gVypjSRJMOPNH-9HS.jpg?quality=95&as=32x30,

# собираем бд из сообществ вк

In [ ]:
#создаем пустой DF для сбора данных
data_posts = pd.DataFrame(columns = ['id_group', 'id_post', 'date', 'description', 'title', 'text', 'views', 'likes', 'reposts'])

In [ ]:
def get_wall_posts(VK_TOKEN: str, owner_id: str, offset: int, count: int, v: float) -> dict:
    url = 'https://api.vk.com/method/wall.get' #метод wall.get позволяет получать записи со стены
    params = {
        'access_token': VK_TOKEN, # ваш токен
        'owner_id': owner_id,
        'v': v,
        'offset': offset,
        'count': count
    }

    response = requests.get(url, params=params)
    return response.json()

In [ ]:
#заменить id сообщества на свое
owner_id = '-15755094', # id сообщества указывается со знаком - перед цифрами
offset = 0, #смещение (сколько постов надо пропустить, чтобы начать сбор)
count = 100 # сколько постов получить. максимум можно получить 100
v = 5.199

wall_posts = get_wall_posts(VK_TOKEN, owner_id, offset, count, v)

#получаем кол-во всех постов в сообществе
count_posts = wall_posts['response']['count']
print('кол-во постов сообщества - ', count_posts)

кол-во постов сообщества -  243835


In [ ]:
posts_number_of_collection = count_posts #кол-во постов для сбора. Если необходимо собрать определенное значение, то заменить count_post на необходимое число (int)
start_post = 0 #счетчик постов откуда будет отсчет сбора данных (начиная с недавних постов)

#если прервалось выполнение кода, то раскоментить эти строчки и запустить заново весь код
#start_post = 104 * 100 #значение post заменить на число из прогрессбара внизу ячейки (кол-во отобранных постов)
#data_posts = pd.read_csv('/content/drive/MyDrive/Data/posts_ria_news.csv') #в кавычках прописать путь где лежит сохраненный частично заполненный DF. Например, '/content/drive/MyDrive/Data/posts_ria_news.csv'

for offset in trange(start_post, posts_number_of_collection, 100):
  wall_posts = get_wall_posts(VK_TOKEN, owner_id, offset, count, v) #получаем данные 100 постов
  id_row_df = len(data_posts) #счетчик строк DF
  for post in range(len(wall_posts['response']['items'])): #проходимся по каждому из 100 постов и вычлиняем информацию из json для записи ее в DF
    #описание и заголовок бывают у ссылок, видео, изображениий.
    #если что то такое есть в посте, то это будут записаны описание и заголовок, если нет, то будет пустая строка
    description, title = '',''
    for element in ['link', 'video', 'photo']:
      try:
        description = wall_posts['response']['items'][post]['attachments'][0][element]['description']
        title = wall_posts['response']['items'][post]['attachments'][0][element]['title']
      except:
        continue
    #эта конструкция нужна для тех постов времен динозавров когда еще не отображалось кол-во просмотров поста
    try:
      views =  int(wall_posts['response']['items'][post]['views']['count'])
    except:
      views = np.nan
    #собираем список необходимой информации для записи в DF
    list_inform = [wall_posts['response']['items'][post]['owner_id'], #id сообщества
                   wall_posts['response']['items'][post]['id'], #id поста
                   wall_posts['response']['items'][post]['date'], #дату поста
                   description, #описание видео, ссылки, изображения если есть
                   title, #заголовок видео, ссылки, изображения если есть
                   wall_posts['response']['items'][post]['text'], #текст поста
                   views, #кол-во просмотров поста
                   wall_posts['response']['items'][post]['likes']['count'], #кол-во лайков
                   wall_posts['response']['items'][post]['reposts']['count'] #кол-во репостов
                   ]
    #если пост есть в датафрейме (проверяем по id поста), то перезаписываем в ту же строчку. кол-во лайков могло со временем измениться
    if wall_posts['response']['items'][post]['id'] in list(data_posts['id_post']):
      index = data_posts[data_posts['id_post'] == wall_posts['response']['items'][post]['id']].index[0] #записываем индекс этого поста в датафрейме
      data_posts.loc[index] = list_inform #перезаписываем строку по найденному индексу
    else: #если поста нет в DF, то записываем его в новую строку
      data_posts.loc[id_row_df] = list_inform
      id_row_df += 1 #обновляем индекс строки для следующих записей в DF
    data_posts.to_csv('/content/drive/MyDrive/Data/posts_ria_news.csv', index = False) #сохраняем DF кажду итерацию, чтобы можно было востановить сбор без потерь данных и времени

  4%|▍         | 104/2439 [20:38<14:10:16, 21.85s/it]